# Start


In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
from graphviz import Source
from IPython.core.magic import register_cell_magic
from IPython.display import HTML, SVG
from matplotlib import pyplot as plt
from scipy.stats import zscore
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    RandomForestClassifier,
)
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    auc,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import (
    GridSearchCV,
    ParameterGrid,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    learning_curve,
    train_test_split,
)
from sklearn.pipeline import FunctionTransformer, Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
)
from sklearn.tree import DecisionTreeClassifier, export_graphviz

## Process data


### Loading data


In [ ]:
file_path: str = "../data/raw"
files: tuple[str, ...] = ("connections", "devices", "processes", "profiles")

dataset: dict[str, pd.DataFrame] = {}
for file in files:
    dataset[file] = pd.read_csv(f"{file_path}/{file}.csv", sep="\t")
    dataset[file] = dataset[file].drop_duplicates()

df = pd.merge(dataset["connections"], dataset["processes"], on=["imei", "ts", "mwra"])
df["ts"] = pd.to_datetime(df.ts)

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

### Cleaning data


In [ ]:
# 1 iteration of cleaning whole dataset of outliers (including p.android.vending)
train_data = train_data[(np.abs(zscore(train_data.iloc[:, 3:])) < 3).all(axis=1)]

# Using all columns except c.android.vending for outlier detection
columns_for_zscore = train_data.iloc[:, 3:].columns.difference(["p.android.vending"])
outliers_count = (~(np.abs(zscore(train_data[columns_for_zscore])) < 3).all(axis=1)).sum()
max_iterations = 10
iteration = 0

# Iterating after we removed all outliers
while outliers_count > 0:
    train_data = train_data[(np.abs(zscore(train_data[columns_for_zscore])) < 3).all(axis=1)]
    outliers_count = (~(np.abs(zscore(train_data[columns_for_zscore])) < 3).all(axis=1)).sum()
    iteration += 1
    if iteration >= max_iterations:
        break

train_data = train_data.reset_index(drop=True)

### Export cleaned data


In [ ]:
# Exporting cleaned data
os.makedirs("../data/clean", exist_ok=True)

train_data.to_csv("../data/clean/train_data.csv", index=False)
test_data.to_csv("../data/clean/test_data.csv", index=False)

### Import cleaned data


In [ ]:
train_data = pd.read_csv("../data/clean/train_data.csv")
test_data = pd.read_csv("../data/clean/test_data.csv")

### Define columns


In [ ]:
all_columns = train_data.drop(columns=["mwra", "ts", "imei"]).columns
non_gaussian_columns = [
    "c.android.vending",
    "c.UCMobile.x86",
    "c.updateassist",
    "c.UCMobile.intl",
    "p.android.vending",
    "p.dogalize",
    "p.olauncher",
    "p.simulator",
    "p.inputmethod.latin",
    "p.android.gms",
    "p.notifier",
    "p.katana",
    "p.gms.persistent",
]
gaussian_columns = all_columns[~all_columns.isin(non_gaussian_columns)]
transformed_feature_order = pd.Series(gaussian_columns.tolist() + non_gaussian_columns)

### Define pipelines


In [ ]:
general_pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("power_transformer", PowerTransformer(method="yeo-johnson")),
    ]
)

vending_pipeline = Pipeline(
    [
        ("quantile_transformer", QuantileTransformer(output_distribution="normal", random_state=42)),
    ]
)

# Create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("general", general_pipe, gaussian_columns),
        ("vending", vending_pipeline, non_gaussian_columns),
    ],
    remainder="passthrough",
)

# Create complete pipeline
complete_pipeline = Pipeline([("preprocessor", preprocessor), ("selector", SelectKBest(f_classif, k=10))])

### Transform data


In [ ]:
# Fit and transform training data, transform test data
train_data_processed = complete_pipeline.fit_transform(train_data[all_columns], train_data["mwra"])
test_data_processed = complete_pipeline.transform(test_data[all_columns])

# Get selected features immediately after fitting
feature_mask = complete_pipeline.named_steps["selector"].get_support()
selected_features = transformed_feature_order[feature_mask]  # order of features is preserved

# Create DataFrames with selected feature names
train_data_processed = pd.DataFrame(train_data_processed, columns=selected_features)
test_data_processed = pd.DataFrame(test_data_processed, columns=selected_features)

train_data_processed["mwra"] = train_data["mwra"]
test_data_processed["mwra"] = test_data["mwra"]

### Export processed data


In [ ]:
os.makedirs("../data/processed", exist_ok=True)

train_data_processed.to_csv("../data/processed/train_data.csv", index=False)
test_data_processed.to_csv("../data/processed/test_data.csv", index=False)

### Cleanup


In [ ]:
del (
    file_path,
    files,
    file,
    df,
    train_data,
    test_data,
    columns_for_zscore,
    outliers_count,
    max_iterations,
    iteration,
    all_columns,
    non_gaussian_columns,
    gaussian_columns,
    transformed_feature_order,
    general_pipe,
    vending_pipeline,
    preprocessor,
    complete_pipeline,
    train_data_processed,
    test_data_processed,
    dataset,
    selected_features,
)

## Helper Stuff


In [ ]:
train_data_processed = pd.read_csv("../data/processed/train_data.csv")
test_data_processed = pd.read_csv("../data/processed/test_data.csv")

# Train data without feature selection
X_train = train_data_processed.drop(columns=["mwra"])
y_train = train_data_processed["mwra"]

# Test data without feature selection
X_test = test_data_processed.drop(columns=["mwra"])
y_test = test_data_processed["mwra"]

del train_data_processed, test_data_processed

In [ ]:
def get_scores(model, model_name, X_train, y_train, X_test, y_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    df = pd.DataFrame(
        index=["accuracy", "precision", "recall", "f1-score", "roc_auc"],
        columns=pd.MultiIndex.from_product([[model_name], ["Train", "Test"]]),
    )

    accuracy_train = accuracy_score(y_train, y_pred_train)
    precision_train = precision_score(y_train, y_pred_train)
    recall_train = recall_score(y_train, y_pred_train)
    f1_train = f1_score(y_train, y_pred_train)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)

    df.loc["accuracy", (model_name, "Train")] = accuracy_train
    df.loc["precision", (model_name, "Train")] = precision_train
    df.loc["recall", (model_name, "Train")] = recall_train
    df.loc["f1-score", (model_name, "Train")] = f1_train
    df.loc["roc_auc", (model_name, "Train")] = roc_auc_train

    accuracy_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test)
    recall_test = recall_score(y_test, y_pred_test)
    f1_test = f1_score(y_test, y_pred_test)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    df.loc["accuracy", (model_name, "Test")] = accuracy_test
    df.loc["precision", (model_name, "Test")] = precision_test
    df.loc["recall", (model_name, "Test")] = recall_test
    df.loc["f1-score", (model_name, "Test")] = f1_test
    df.loc["roc_auc", (model_name, "Test")] = roc_auc_test

    return df

In [ ]:
def get_scores_cv(model, model_name, X, y, cv):
    metrics = ["accuracy", "precision", "recall", "f1", "roc_auc"]

    # Create MultiIndex DataFrame
    df = pd.DataFrame(
        index=pd.MultiIndex.from_product([["Train", "Test"], metrics]),
        columns=pd.MultiIndex.from_product([[model_name], ["Mean", "Std"]]),
    )

    # Calculate scores for each model
    scores = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring={
            "accuracy": "accuracy",
            "precision": "precision",
            "recall": "recall",
            "f1": "f1",
            "roc_auc": "roc_auc",
        },
        return_train_score=True,
    )

    # Fill DataFrame with training and test scores
    for metric in metrics:
        # Training scores
        train_key = f"train_{metric}"
        df.loc[("Train", metric), (model_name, "Mean")] = scores[train_key].mean()
        df.loc[("Train", metric), (model_name, "Std")] = scores[train_key].std()

        # Test scores
        test_key = f"test_{metric}"
        df.loc[("Test", metric), (model_name, "Mean")] = scores[test_key].mean()
        df.loc[("Test", metric), (model_name, "Std")] = scores[test_key].std()

    return df

In [ ]:
def plot_learning_curve(estimator, X, y, train_sizes, cv, scoring, title):
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, train_sizes=train_sizes, scoring=scoring, n_jobs=-1
    )
    train_scores_mean = 1 - train_scores.mean(axis=1)
    test_scores_mean = 1 - test_scores.mean(axis=1)

    plt.figure()
    plt.plot(train_sizes, train_scores_mean, label="Training score")
    plt.plot(train_sizes, test_scores_mean, label="Cross-Validation Score")
    plt.xlabel("Training Size")
    plt.ylabel(f"{scoring} Error")
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
def plot_model_complexity_curve(model, X_train, y_train, X_test, y_test, max_depth_range):
    # Initialize lists to store the training and validation errors
    train_errors = []
    val_errors = []

    # Loop over the range of max_depth values
    for max_depth in max_depth_range:
        # Initialize the model with the current max_depth
        model = DecisionTreeClassifier(
            criterion="gini",
            max_depth=max_depth,
            min_samples_split=10,
            min_samples_leaf=1,
            ccp_alpha=0.001,
            random_state=42,
        )

        # Fit model
        model.fit(X_train, y_train)

        # Compute the training and testing data errors
        train_score = accuracy_score(y_train, model.predict(X_train))
        test_score = accuracy_score(y_test, model.predict(X_test))

        # Compute the mean errors
        train_errors.append(1 - train_score)
        val_errors.append(1 - test_score)

    # Plot the learning curve for model complexity
    plt.figure()
    plt.plot(max_depth_range, train_errors, label="Training Error")
    plt.plot(max_depth_range, val_errors, label="Validation Error")
    plt.xlabel("Max Depth")
    plt.ylabel("Error")
    plt.title("Learning Curve (Model Complexity)")
    plt.legend()
    plt.show()

In [ ]:
def compare_confusion_matrix(models, X_train, y_train, X_test, y_test):
    _, axes = plt.subplots(1, len(models), figsize=(15, 5))

    for i, (model_name, model) in enumerate(models):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        conf_matrix = confusion_matrix(y_test, y_pred)
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False, ax=axes[i])
        axes[i].set_title(model_name)

    plt.show()

In [ ]:
def show_confusion_matrix(model, model_name, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(model_name)
    plt.show()

In [ ]:
@register_cell_magic
def ignore(line, cell):
    pass

# 3.1


-   Jednoduchý klasifikátor na základe závislosti v dátach.


## A


Naimplementujte jednoduchý ID3 klasifikátor s hĺbkou min 2 (vrátane root/koreň).


In [ ]:
class Node:
    def __init__(self, attribute=None, value=None, is_leaf=False, prediction=None):
        self.attribute = attribute
        self.value = value
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.children = {}


class ID3Classifier:
    def __init__(self, min_depth: int = 2, max_depth: int = None) -> None:
        self.max_depth = max_depth
        self.min_depth = min_depth
        self.tree = None

    def fit(self, X, y):
        data = X.copy()
        data["label"] = y
        self.tree = self._build_tree(data, current_depth=0)

    def _entropy(self, y):
        proportions = y.value_counts(normalize=True)
        return -np.sum(proportions * np.log2(proportions))

    def _information_gain(self, data, attribute, target="label"):
        total_entropy = self._entropy(data[target])
        values = data[attribute].unique()
        weighted_entropy = 0
        for value in values:
            subset = data[data[attribute] == value]
            weighted_entropy += (len(subset) / len(data)) * self._entropy(subset[target])
        return total_entropy - weighted_entropy

    def _best_split(self, data, attributes, target="label"):
        best_gain = -1
        best_attribute = None
        for attribute in attributes:
            gain = self._information_gain(data, attribute, target)
            if gain > best_gain:
                best_gain = gain
                best_attribute = attribute
        return best_attribute

    def _build_tree(self, data, current_depth):
        X = data.drop(columns=["label"])
        y = data["label"]

        if len(y.unique()) == 1 or (self.max_depth is not None and current_depth >= self.max_depth):
            return Node(is_leaf=True, prediction=y.iloc[0])

        if current_depth >= self.min_depth or X.empty:
            return Node(is_leaf=True, prediction=y.mode().iloc[0])

        best_attribute = self._best_split(data, X.columns)
        root = Node(attribute=best_attribute)

        for value in data[best_attribute].unique():
            subset = data[data[best_attribute] == value]
            root.children[value] = self._build_tree(subset, current_depth + 1)

        return root

    def predict(self, X):
        return X.apply(self._predict_row, axis=1, args=(self.tree,))

    def _predict_row(self, row, node):
        if node.is_leaf:
            return node.prediction

        attribute_value = row[node.attribute]
        if attribute_value in node.children:
            return self._predict_row(row, node.children[attribute_value])

        return None

### Not using feature selection


In [ ]:
# Initialize the ID3 classifier
id3_classifier = ID3Classifier(min_depth=2, max_depth=5)
id3_classifier.fit(X_train, y_train)

In [ ]:
graph = Source(
    export_graphviz(
        id3_classifier, out_file=None, class_names=["no", "yes"], filled=True, feature_names=X_train.columns
    )
)

display(SVG(graph.pipe(format="svg")))

style = "<style>svg{width:100%;height:70%;}</style>"
HTML(style)

## B


Vyhodnoťte Váš ID3 klasifikátor pomocou metrík accuracy, precision a recall.


In [ ]:
df_1 = get_scores(dtc_nofs, "DecisionTreeClassifier - No feature selection", X_train, y_train, X_test, y_test)
df_2 = get_scores(
    dtc_fs, "DecisionTreeClassifier - Feature selection", X_train_selected, y_train, X_test_selected, y_test
)

df = pd.concat([df_1, df_2], axis=1)
del df_1, df_2
df

-   We see that using feature selection gives worse results. We will look at it in more detail later (3.4).


## C


Zístite či Váš ID3 klasifikátor má overfit.


-   Since train and test metrics are close, the model is likely not overfitting.
-   But let's also look at learning curves.


### Learning curve - train sizes


In [ ]:
# Initialize the model
dtc_nofs = DecisionTreeClassifier(
    criterion="gini", max_depth=15, min_samples_split=10, min_samples_leaf=1, ccp_alpha=0.001, random_state=42
)

# Define the training sizes
train_sizes = np.linspace(0.1, 1.0, 50)

# Plot learning curve for accuracy
plot_learning_curve(
    dtc_nofs,
    X_train,
    y_train,
    train_sizes,
    cv=5,
    scoring="accuracy",
    title="Learning Curve - Train size (Accuracy)",
)

# Plot learning curve for precision
plot_learning_curve(
    dtc_nofs,
    X_train,
    y_train,
    train_sizes,
    cv=5,
    scoring="precision",
    title="Learning Curve - Train size (Precision)",
)

# Plot learning curve for recall
plot_learning_curve(
    dtc_nofs, X_train, y_train, train_sizes, cv=5, scoring="recall", title="Learning Curve - Train size (Recall)"
)

-   We don't see a big gap in metrics between train and test data (Looking at last point since that is what we used in previous steps).
-   However we see that model starts of badly (as expected) and then improves with more data. Around 50% of data for training seems to be enough.


### Learning curve - model complexity


In [ ]:
# Define the range of max_depth values
max_depth_range = range(1, 30)

# Plot model complexity curve
plot_model_complexity_curve(dtc_nofs, X_train, y_train, X_test, y_test, max_depth_range)

-   This also shows no overfitting for our graph as there is no significant gap (looking at max_depth=15, as this is what we used in previous steps).
-   We can also see that model start of very badly and is underfitting.
-   Around max_depth=5 training and testing error stabilizes.


### ROC


In [ ]:
dtc_nofs = DecisionTreeClassifier(
    criterion="gini", max_depth=15, min_samples_split=10, min_samples_leaf=1, ccp_alpha=0.001, random_state=42
)

dtc_nofs.fit(X_train, y_train)

# Compute ROC curve and ROC area for train data
fpr_train, tpr_train, _ = roc_curve(y_train, dtc_nofs.predict_proba(X_train)[:, 1])
roc_auc_train = auc(fpr_train, tpr_train)

# Compute ROC curve and ROC area for test data
fpr_test, tpr_test, _ = roc_curve(y_test, dtc_nofs.predict_proba(X_test)[:, 1])
roc_auc_test = auc(fpr_test, tpr_test)

# Plot ROC curve
plt.figure()
plt.plot(fpr_train, tpr_train, color="blue", lw=2, label=f"Train ROC curve (area = {roc_auc_train:.5f})")
plt.plot(fpr_test, tpr_test, color="red", lw=2, label=f"Test ROC curve (area = {roc_auc_test:.5f})")
plt.plot([0, 1], [0, 1], color="gray", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.show()

-   ROC also shows no significant gap between train and test data. So we can conclude that model is not overfitting.


## Cleanup


In [ ]:
del (
    dtc_fs,
    dtc_nofs,
    fpr_train,
    tpr_train,
    fpr_test,
    tpr_test,
    train_sizes,
    max_depth_range,
    roc_auc_test,
    roc_auc_train,
)

# 3.2


Trénovanie a vyhodnotenie klasifikátorov strojového učenia.


## A


Na trénovanie využite jeden stromový algoritmus v scikit-learn.


### Initialize DecisionTreeClassifier


In [ ]:
# use some basic parameters to avoid overfitting
dst_classifier = DecisionTreeClassifier(max_depth=15, ccp_alpha=0.001, random_state=42)
dst_classifier.fit(X_train, y_train)

### Metrics


In [ ]:
df_1 = get_scores_cv(dst_classifier, "DecisionTreeClassifier", X_train, y_train, cv=5)
df_1

In [ ]:
show_confusion_matrix(dst_classifier, "RandomForestClassifier", X_train, y_train, X_test, y_test)

-   Even quite simple model gives good results without overfitting.


## B


Porovnajte s jedným iným nestromovým algoritmom v scikit-learn.


### Initialize LogisticRegression


In [ ]:
log_reg = LogisticRegression(random_state=42)

### Metrics


In [ ]:
df_2 = get_scores_cv(log_reg, "LogisticRegression", X_train, y_train, cv=5)
df_2

In [ ]:
show_confusion_matrix(log_reg, "LogisticRegression", X_train, y_train, X_test, y_test)

-   Basic logistic regression gives quite good results without overfitting as well


### Comparison


In [ ]:
df = pd.concat([df_1, df_2], axis=1)
del df_1, df_2
df

In [ ]:
compare_confusion_matrix(
    [("RandomForestClassifier", dst_classifier), ("LogisticRegression", log_reg)], X_train, y_train, X_test, y_test
)

-   Basic Logistic Regression gives slightly better results than basic Decision Tree Classifier.


## C


Porovnajte výsledky s ID3 z prvého kroku.


In [ ]:
# TODO: Finish

## D


Vizualizujte natrénované pravidlá minimálne pre jeden Vami vybraný algoritmus.


In [ ]:
# Initialize RandomForestClassifier
dst_classifier = DecisionTreeClassifier(max_depth=15, ccp_alpha=0.001, random_state=42)

# Fit the classifier to the training data
dst_classifier.fit(X_train, y_train)

# Generate graph
graph = Source(
    export_graphviz(
        dst_classifier, out_file=None, class_names=["no", "yes"], filled=True, feature_names=X_train.columns
    )
)

# Display graph
display(SVG(graph.pipe(format="svg")))

style = "<style>svg{width:100%;height:70%;}</style>"
HTML(style)

## E


Vyhodnoťte natrénované modely pomocou metrík accuracy, precision a recall.


-   We already did this but let's do it again.


### Initialize classifiers


In [ ]:
# Initialize classifiers
dst_classifier = DecisionTreeClassifier(max_depth=15, ccp_alpha=0.001, random_state=42)
log_reg = LogisticRegression(random_state=42)

### Compare metrics


In [ ]:
# Print the scores
df_1 = get_scores_cv(dst_classifier, "DecisionTreeClassifier", X_train, y_train, cv=5)
df_2 = get_scores_cv(log_reg, "LogisticRegression", X_train, y_train, cv=5)

df = pd.concat([df_1, df_2], axis=1)
del df_1, df_2
df

### Compare matrixes


In [ ]:
compare_confusion_matrix(
    [
        ("DecisionTreeClassifier", dst_classifier),
        ("LogisticRegression", log_reg),
    ],
    X_train,
    y_train,
    X_test,
    y_test,
)

### Comparison


-   As we already said, Logistic Regression gives slightly better results than Decision Tree Classifier.
-   However, both models are quite good and don't overfit.


## Cleanup


In [ ]:
del dst_classifier, log_reg, graph, style

# 3.3


Optimalizácia alias hyperparameter tuning.


## A


Vyskúšajte rôzne nastavenie hyperparametrov (tuning) pre zvolený algoritmus tak,
aby ste optimalizovali výkonnosť (bez underfitingu).

We will test parameters for RandomForestClassifier as we know this will be better than DecisionTreeClassifier.


### Decision Tree vs Random Forest


In [ ]:
%%ignore
# Define the parameter grid
param_grid = {
    "max_depth": [5, 10, 15, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "ccp_alpha": [0.0, 0.0005, 0.001, 0.0015],
    "min_impurity_decrease": [0.0, 0.0005, 0.001, 0.0015],
}

# Initialize the DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier, param_grid=param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=1
)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

-   Best Parameters: {'ccp_alpha': 0.0005, 'max_depth': 10, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 4, 'min_samples_split': 2}
-   Best Score: 0.9026635738931255


In [ ]:
dt_classifier = DecisionTreeClassifier(
    max_depth=10, min_samples_split=2, min_samples_leaf=4, ccp_alpha=0.0005, random_state=42
)
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=15, ccp_alpha=0.001, random_state=42)

df_1 = get_scores_cv(dt_classifier, "DecisionTreeClassifier", X_train, y_train, cv=5)
df_2 = get_scores_cv(rf_classifier, "RandomForestClassifier", X_train, y_train, cv=5)

df = pd.concat([df_1, df_2], axis=1)
del df_1, df_2
df

-   This shows difference between optimized Decision Tree and some basic Random Forest (with basic paramters to avoid overfitting).
-   We can clearly see that Random Forest is better and therefore we are gonna use it for further steps.


### First, find baseline parameters


#### n_estimators


In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    n_estimators=100, min_samples_split=2, min_samples_leaf=2, max_depth=20, ccp_alpha=0.001
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
df = get_scores(rf_classifier, "RandomForestClassifier", X_train, y_train, X_test, y_test)
df

In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_depth=20, ccp_alpha=0.001
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
df = get_scores(rf_classifier, "RandomForestClassifier", X_train, y_train, X_test, y_test)
df

-   We see that performance is not increasing with n_estimators.


#### ccp_alpha


In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(criterion="entropy", n_estimators=100, random_state=42, ccp_alpha=0.002)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
df = get_scores(rf_classifier, "RandomForestClassifier", X_train, y_train, X_test, y_test)
df

In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(criterion="entropy", n_estimators=100, random_state=42, ccp_alpha=0.005)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
df = get_scores(rf_classifier, "RandomForestClassifier", X_train, y_train, X_test, y_test)
df

-   We see ccp_alpha 0.002 is better considering ROC AUC factor.


#### max_features


In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    criterion="entropy",
    n_estimators=100,
    random_state=42,
    ccp_alpha=0.002,
    max_features="sqrt",
    max_depth=7,
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
df = get_scores(rf_classifier, "RandomForestClassifier", X_train, y_train, X_test, y_test)
df

In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    criterion="entropy",
    n_estimators=100,
    random_state=42,
    ccp_alpha=0.002,
    max_features="log2",
    max_depth=7,
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
df = get_scores(rf_classifier, "RandomForestClassifier", X_train, y_train, X_test, y_test)
df

-   When also using max_depth, there is difference in ROC AUC, sqrt is better.


#### criterion


In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    criterion="entropy",
    n_estimators=100,
    random_state=42,
    ccp_alpha=0.002,
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
df = get_scores(rf_classifier, "RandomForestClassifier", X_train, y_train, X_test, y_test)
df

In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    criterion="gini",
    n_estimators=100,
    random_state=42,
    ccp_alpha=0.002,
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
df = get_scores(rf_classifier, "RandomForestClassifier", X_train, y_train, X_test, y_test)
df

-   Entropy is better considering ROC AUC factor.


### RandomizedSearchCV


-   In previous step, we found that n_estimators=100, ccp_alpha=0.002, max_features=sqrt, criterion=entropy are better.
-   However we are not gonna look at max_features and criterion in first tuning method as we will look at them at the end when we have best primary hyperparameters.
-   Not using max_features and criterion in first tuning gives more priority to other hyperparameters.
-   Our primary hyperparameters are n_estimators, max_depth, min_samples_split, min_samples_leaf, ccp_alpha.


In [ ]:
%%ignore
# Define the broad parameter grid
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, 15, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "ccp_alpha": [0.001, 0.002, 0.003],
}

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_classifier,
    param_distributions=param_grid,
    n_iter=100,
    n_jobs=-1,
    verbose=1,
    cv=10,  # 10-fold cross-validation for more reliable results
    scoring="roc_auc",
    random_state=42,
)

# Fit the classifier to the training data
random_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Score: {best_score:.5f}")

-   Best Parameters: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 15, 'ccp_alpha': 0.002}
-   Best Score: 0.91579


### GridSearchCV


#### First iteration


In [ ]:
%%ignore
# Best parameters from RandomizedSearchCV
best_params = {
    "n_estimators": 300,
    "max_depth": 15,
    "min_samples_split": 10,
    "min_samples_leaf": 2,
    "ccp_alpha": 0.002,
}

# Define the refined parameter grid for GridSearchCV
param_grid_refined = {
    "n_estimators": [
        best_params["n_estimators"] - 100,
        best_params["n_estimators"],
        best_params["n_estimators"] + 100,
    ],
    "max_depth": [
        best_params["max_depth"] - 5,
        best_params["max_depth"],
        best_params["max_depth"] + 5,
    ],
    "min_samples_split": [
        best_params["min_samples_split"] - 5,
        best_params["min_samples_split"],
        best_params["min_samples_split"] + 5,
    ],
    "min_samples_leaf": [
        best_params["min_samples_leaf"],
        best_params["min_samples_leaf"] + 1,
    ],
    "ccp_alpha": [
        best_params["ccp_alpha"],
        best_params["ccp_alpha"] + 0.001,
        best_params["ccp_alpha"] + 0.002,
    ],
}

# Initialize the RandomForestClassifier with the best parameters from RandomizedSearchCV
rf_refined = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_refined = GridSearchCV(
    estimator=rf_refined,
    param_grid=param_grid_refined,
    verbose=1,
    n_jobs=-1,
    cv=10,  # 10-fold cross-validation for more reliable results
    scoring="roc_auc",
)

# Fit the classifier to the training data
grid_search_refined.fit(X_train, y_train)

# Get the best parameters and best score
best_params_refined = grid_search_refined.best_params_
best_score_refined = grid_search_refined.best_score_

print(f"Best Parameters after Refining: {best_params_refined}")
print(f"Best Score after Refining: {best_score_refined:.5f}")

-   Best Parameters after Refining: {'ccp_alpha': 0.002, 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 15, 'n_estimators': 400}
-   Best Score after Refining: 0.91606
-   The trend for n_estimators is, bigger is better (until some point). So in next iteration we will ignore this parameter.


#### Second iteration


My colleague found different parameters to be better, so we will look at them in this step.


In [ ]:
%%ignore
# Define the refined parameter grid for GridSearchCV
param_grid_refined = {
    "n_estimators": [
        200
    ],
    "max_depth": [
        15
    ],
    "min_samples_split": [
        5,10
    ],
    "min_samples_leaf": [
        1,2
    ],
    "ccp_alpha": [
        0.001, 0.002
    ],
}

# Initialize the RandomForestClassifier with the best parameters from RandomizedSearchCV
rf_refined = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_refined = GridSearchCV(
    estimator=rf_refined,
    param_grid=param_grid_refined,
    verbose=1,
    n_jobs=-1,
    cv=10,  # 10-fold cross-validation for more reliable results
    scoring="roc_auc",
)

# Fit the classifier to the training data
grid_search_refined.fit(X_train, y_train)

# Get the best parameters and best score
best_params_refined = grid_search_refined.best_params_
best_score_refined = grid_search_refined.best_score_

print(f"Best Parameters after Refining: {best_params_refined}")
print(f"Best Score after Refining: {best_score_refined:.5f}")

-   Best Parameters after Refining: {'ccp_alpha': 0.001, 'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
-   Best Score after Refining: 0.91596
-   From these observations:
    -   We can conclude that max_depth=~15 is the best, as it was 15 in every iteration.
    -   We can conclude that ccp_alpha=~0.001 or cpp_alpha=~0.002 is the best.
    -   We can conclude that min_samples_leaf=~1 or a little higher is the best.
    -   We can conclude that min_samples_split=~10 is the best.
    -   We can conclude that higher n_estimators is better (at some value it will be worse).


#### Third iteration


Lets look at cpp_alpha and max_depth in more detail.


In [ ]:
%%ignore
# Define the refined parameter grid for GridSearchCV
param_grid_refined = {
    "n_estimators": [300],
    "max_depth": [14, 15, 16],
    "min_samples_split": [10],
    "min_samples_leaf": [1],
    "ccp_alpha": [0.0095, 0.01, 0.0105],
}

# Initialize the RandomForestClassifier with the best parameters from RandomizedSearchCV
rf_refined = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_refined = GridSearchCV(
    estimator=rf_refined,
    param_grid=param_grid_refined,
    verbose=1,
    n_jobs=-1,
    cv=10,  # 10-fold cross-validation for more reliable results
    scoring="roc_auc",
)

# Fit the classifier to the training data
grid_search_refined.fit(X_train, y_train)

# Get the best parameters and best score
best_params_refined = grid_search_refined.best_params_
best_score_refined = grid_search_refined.best_score_

print(f"Best Parameters after Refining: {best_params_refined}")
print(f"Best Score after Refining: {best_score_refined:.5f}")

-   cpp_alpha=0.001 is best
-   max_depth=15 is best


#### Fourth iteration


Now let's look at best n_estimators.


In [ ]:
%%ignore
# Define the refined parameter grid for GridSearchCV
param_grid_refined = {
    "n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    "max_depth": [15],
    "min_samples_split": [10],
    "min_samples_leaf": [1],
    "ccp_alpha": [0.001],
}

# Initialize the RandomForestClassifier with the best parameters from RandomizedSearchCV
rf_refined = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_refined = GridSearchCV(
    estimator=rf_refined,
    param_grid=param_grid_refined,
    verbose=1,
    n_jobs=-1,
    cv=10,  # 10-fold cross-validation for more reliable results
    scoring="roc_auc",
)

# Fit the classifier to the training data
grid_search_refined.fit(X_train, y_train)

# Get the best parameters and best score
best_params_refined = grid_search_refined.best_params_
best_score_refined = grid_search_refined.best_score_

print(f"Best Parameters after Refining: {best_params_refined}")
print(f"Best Score after Refining: {best_score_refined:.5f}")

-   n_estimators=300 is best.


#### Final iteration


As discussed earlier, we will also look at max_features and criterion.


In [ ]:
%%ignore
# Define the refined parameter grid for GridSearchCV
param_grid_refined = {
    "n_estimators": [300],
    "max_depth": [15],
    "min_samples_split": [10],
    "min_samples_leaf": [1],
    "ccp_alpha": [0.001],
    "max_features": ["sqrt", "log2"],
    "criterion": ["entropy", "gini"],
}

# Initialize the RandomForestClassifier with the best parameters from RandomizedSearchCV
rf_refined = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_refined = GridSearchCV(
    estimator=rf_refined,
    param_grid=param_grid_refined,
    verbose=1,
    n_jobs=-1,
    cv=10,  # 10-fold cross-validation for more reliable results
    scoring="roc_auc",
)

# Fit the classifier to the training data
grid_search_refined.fit(X_train, y_train)

# Get the best parameters and best score
best_params_refined = grid_search_refined.best_params_
best_score_refined = grid_search_refined.best_score_

print(f"Best Parameters after Refining: {best_params_refined}")
print(f"Best Score after Refining: {best_score_refined:.5f}")

-   Best Parameters after Refining: {'ccp_alpha': 0.001, 'criterion': 'gini', 'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
-   Best Score after Refining: 0.91632
-   We can see that max_features=sqrt and criterion=gini are best and they are also default values.


Our final parameters are:

-   n_estimators=300
-   max_depth=15
-   min_samples_split=10
-   min_samples_leaf=1
-   ccp_alpha=0.001
-   max_features="sqrt"
-   criterion="gini"


To note we did these test without having feature selection in pipeline as we were getting better result without it. (Will be discussed further in 3.4)

Since we discovered that it is best practise to always use feature selection, we will use it it next step and look if parameters change.


In [ ]:
# Define the refined parameter grid for GridSearchCV
param_grid_refined = {
    "n_estimators": [300],
    "max_depth": [10, 15],
    "min_samples_split": [5, 10],
    "min_samples_leaf": [1, 2],
    "ccp_alpha": [0.001, 0.0015],
}

# Initialize the RandomForestClassifier with the best parameters from RandomizedSearchCV
rf_refined = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_refined = GridSearchCV(
    estimator=rf_refined,
    param_grid=param_grid_refined,
    verbose=1,
    n_jobs=-1,
    cv=10,  # 10-fold cross-validation for more reliable results
    scoring="roc_auc",
)

# Fit the classifier to the training data
grid_search_refined.fit(X_train, y_train)

# Get the best parameters and best score
best_params_refined = grid_search_refined.best_params_
best_score_refined = grid_search_refined.best_score_

print(f"Best Parameters after Refining: {best_params_refined}")
print(f"Best Score after Refining: {best_score_refined:.5f}")

-   Best Parameters after Refining: {'ccp_alpha': 0.001, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
-   Best Score after Refining: 0.91591
-   Best score without feature selection was 0.91632 vs 0.91591 with feature selection.
-   As hinted before, using feature selection gives slightly worse results. But we will look at it in more detail in 3.4.
-   We can also see that parameters did change and therefore it is always crucial to try different parameters when changing pipeline.


Our final parameters with feature selection are:

-   n_estimators=300
-   max_depth=10
-   min_samples_split=5
-   min_samples_leaf=1
-   ccp_alpha=0.001
-   max_features="sqrt"
-   criterion="gini"


### GridSearchCV - Logistic Regression


-   Let's also look at Logistic Regression.


In [ ]:
%%ignore
log_reg = LogisticRegression(solver="lbfgs", random_state=42)

parameters = {
    "penalty": [None, "l2"],
    "C": [0.001, 0.005, 0.01, 0.05],
    "max_iter": [100, 150, 200, 250, 300],
    "tol": [0.0001, 0.0005, 0.001, 0.005],
}

grid_search = GridSearchCV(
    log_reg,
    param_grid=parameters,
    scoring="roc_auc",
    n_jobs=-1,
    cv=10,
)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Score: {best_score:.5f}")

-   Best Parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', 'tol': 0.0005}
-   Best Score: 0.91217


In [ ]:
%%ignore
log_reg = LogisticRegression(solver="liblinear", random_state=42)

parameters = {
    "penalty": ["l1", "l2"],
    "C": [0.001, 0.005, 0.01, 0.05],
    "max_iter": [100, 150, 200, 250, 300],
    "tol": [0.0001, 0.0005, 0.001, 0.005],
}

grid_search = GridSearchCV(
    log_reg,
    param_grid=parameters,
    scoring="roc_auc",
    n_jobs=-1,
    cv=10,
)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Score: {best_score:.5f}")

-   Best Parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', 'tol': 0.001}
-   Best Score: 0.91243


In [ ]:
log_reg = LogisticRegression(C=0.01, max_iter=100, penalty="l2", solver="lbfgs", tol=0.001, random_state=42)
log_reg.fit(X_train, y_train)
df_1 = get_scores(log_reg, "LogisticRegression - lbfgs", X_train, y_train, X_test, y_test)

log_reg = LogisticRegression(C=0.01, max_iter=100, penalty="l1", solver="liblinear", tol=0.001, random_state=42)
log_reg.fit(X_train, y_train)
df_2 = get_scores(log_reg, "LogisticRegression - liblinear", X_train, y_train, X_test, y_test)

# Concatenate the DataFrames
df = pd.concat([df_1, df_2], axis=1)
df

-   Best Parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', solver="lbfgs",'tol': 0.001}


## B


Vyskúšajte kombinácie modelov (ensemble) pre zvolený algoritmus tak, aby ste
optimalizovali výkonnosť (bez underfitingu).


We already used RandomForestClassifier in previous step, but we will now explore more ensemble methods.


### Best params


In [ ]:
rf_best_params = {
    "n_estimators": 300,
    "max_depth": 10,
    "min_samples_split": 5,
    "min_samples_leaf": 1,
    "ccp_alpha": 0.001,
    "random_state": 42,
}

lgr_best_params = {"C": 0.01, "max_iter": 100, "penalty": "l2", "solver": "lbfgs", "tol": 0.001}

hbg_best_params = {
    "l2_regularization": 2.0,
    "learning_rate": 0.1,
    "max_depth": 7,
    "max_iter": 200,
    "min_samples_leaf": 20,
    "random_state": 42,
}

### Basic Ensemble Models


#### RandomForestClassifier


In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    **rf_best_params,
)

# Print the scores
df_rf = get_scores_cv(rf_classifier, "RandomForestClassifier", X_train, y_train, cv=5)
df_rf

#### GradientBoostingClassifier


In [ ]:
# Initialize the GradientBoostingClassifier
gb_classifier = GradientBoostingClassifier(random_state=42)

# Print the scores
df_gb = get_scores_cv(gb_classifier, "GradientBoostingClassifier", X_train, y_train, cv=5)
df_gb

#### HistGradientBoostingClassifier


In [ ]:
hgb_classifier = HistGradientBoostingClassifier(random_state=42)

# Print the scores
df_hgb = get_scores_cv(hgb_classifier, "HistGradientBoostingClassifier", X_train, y_train, cv=5)
df_hgb

#### Comparisson


In [ ]:
df = pd.concat([df_rf, df_gb, df_hgb], axis=1)
df

-   We can see that RandomForestClassifier is the best model for our data as we used hyperparameter tuning on it.
-   HistGradientBoostingClassifier shows promising results.


#### Trying to tune HistGradientBoostingClassifier


In [ ]:
# Parameter grid
param_grid = {
    "learning_rate": [0.01, 0.1, 0.3],
    "max_depth": [3, 5, 7],
    "max_iter": [100, 200],
    "min_samples_leaf": [20, 50],
    "l2_regularization": [0, 1.0, 2.0],
}

# Base classifier
hgb = HistGradientBoostingClassifier(random_state=42)

# Grid search with 5-fold CV
grid_search = GridSearchCV(estimator=hgb, param_grid=param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=1)

# Assuming X and y are your features and target
grid_search.fit(X_train, y_train)

# Get best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

# Create optimized model with best parameters
best_model = HistGradientBoostingClassifier(**grid_search.best_params_, random_state=42)

In [ ]:
hgb = HistGradientBoostingClassifier(
    l2_regularization=2.0, learning_rate=0.1, max_depth=7, max_iter=200, min_samples_leaf=20, random_state=42
)

df_hgb = get_scores_cv(hgb, "HistGradientBoostingClassifier - Refined", X_train, y_train, cv=5)
df = pd.concat([df_rf, df_hgb], axis=1)
df

-   We can see that RandomForestClassifier is still the best model for our data as we used hyperparameter tuning on it.


### Voting Classifier


In [ ]:
from sklearn.ensemble import (
    HistGradientBoostingClassifier,
    RandomForestClassifier,
    StackingClassifier,
    VotingClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Define base models with different algorithms
rf = RandomForestClassifier(**rf_best_params)

lgr = LogisticRegression(**lgr_best_params)

hgb = HistGradientBoostingClassifier(**hbg_best_params)

# Create voting ensemble with different algorithms
voting_clf = VotingClassifier(estimators=[("rf", rf), ("lgr", lgr), ("hgb", hgb)], n_jobs=-1, voting="soft")

# Create stacking ensemble
stacking_clf = StackingClassifier(
    estimators=[("rf", rf), ("lgr", lgr), ("hgb", hgb)], final_estimator=LogisticRegression(random_state=42), cv=5
)

# Evaluate all models
models = {
    "Random Forest": rf,
    "Logistic Regression": lgr,
    "HistGradient Boosting": hgb,
    "Voting Ensemble": voting_clf,
    "Stacking Ensemble": stacking_clf,
}

df = pd.DataFrame()
for name, model in models.items():
    print(f"\nEvaluating {name}")
    df_tmp = get_scores_cv(model, name, X_train, y_train, cv=5)
    df = pd.concat([df, df_tmp], axis=1)
df

-   Main finding is that Voting Classifier/Stacking Ensemble is better than sole RandomForestClassifier.
-   There are only slight differences between Voting Classifier and Stacking Ensemble.
-   Only concern using Voting Ensemble is difference between train/test score for roc_auc, but since other metrics are close there might not be overfitting.


## C


Využite krížovú validáciu (cross validation) na trénovacej množine.


-   We were already using cross validation in previous steps.


## D


Dokážte že Váš nastavený najlepší model je bez overfitingu.


In [ ]:
# Define base models with different algorithms
rf = RandomForestClassifier(**rf_best_params)

lgr = LogisticRegression(**lgr_best_params)

hgb = HistGradientBoostingClassifier(**hbg_best_params)

# Create voting ensemble with different algorithms
voting_clf = VotingClassifier(estimators=[("rf", rf), ("lgr", lgr), ("hgb", hgb)], n_jobs=-1, voting="soft")

voting_clf.fit(X_train, y_train)

df = get_scores(voting_clf, "VotingClassifier", X_train, y_train, X_test, y_test)
df

-   Since train metrics and test metrics are close, the model is not overfitting.


# 3.4


Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasifikáciu.


## A


Stratégie riešenia chýbajúcich hodnôt a outlierov.


### Definitions


In [ ]:
def load_data_raw():
    file_path: str = "../data/raw"
    files: tuple[str, ...] = ("connections", "devices", "processes", "profiles")

    dataset: dict[str, pd.DataFrame] = {}
    for file in files:
        dataset[file] = pd.read_csv(f"{file_path}/{file}.csv", sep="\t")
        dataset[file] = dataset[file].drop_duplicates()

    df = pd.merge(dataset["connections"], dataset["processes"], on=["imei", "ts", "mwra"])
    df["ts"] = pd.to_datetime(df.ts)

    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
    train_data = train_data.reset_index(drop=True)
    test_data = test_data.reset_index(drop=True)

    return train_data, test_data

In [ ]:
# Load data
train_data, test_data = load_data_raw()

# Define columns
all_columns = train_data.drop(columns=["mwra", "ts", "imei"]).columns
non_gaussian_columns = [
    "c.android.vending",
    "c.UCMobile.x86",
    "c.updateassist",
    "c.UCMobile.intl",
    "p.android.vending",
    "p.dogalize",
    "p.olauncher",
    "p.simulator",
    "p.inputmethod.latin",
    "p.android.gms",
    "p.notifier",
    "p.katana",
    "p.gms.persistent",
]
gaussian_columns = all_columns[~all_columns.isin(non_gaussian_columns)]
transformed_feature_order = pd.Series(gaussian_columns.tolist() + non_gaussian_columns)

In [ ]:
def remove_outliers_iqr(data, columns):
    Q1 = data[columns].quantile(0.25)
    Q3 = data[columns].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[columns] >= lower_bound).all(axis=1) & (data[columns] <= upper_bound).all(axis=1)]

In [ ]:
general_pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("power_transformer", PowerTransformer(method="yeo-johnson")),
    ]
)

vending_pipeline = Pipeline(
    [
        ("quantile_transformer", QuantileTransformer(output_distribution="normal", random_state=42)),
    ]
)

# Create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("general", general_pipe, gaussian_columns),
        ("vending", vending_pipeline, non_gaussian_columns),
    ],
    remainder="passthrough",
)

# Create complete pipeline
complete_pipeline = Pipeline([("preprocessor", preprocessor)])

In [ ]:
number_of_outliers = pd.DataFrame(
    index=["Number of Outliers", "Percentage of Outliers"],
    columns=["None-iterative IQR", "Iterative IQR", "None-iterative Z-score", "Iterative Z-score"],
)

### Cleaning methods


#### None-iterative IQR


##### Cleaning


In [ ]:
# Load data
train_data, test_data = load_data_raw()

# Get number of rows before removing outliers
number_of_rows_before_outliers = train_data.shape[0]

# Remove outliers using IQR method
train_data = remove_outliers_iqr(train_data, train_data.iloc[:, 3:].columns)

# Reset index
train_data = train_data.reset_index(drop=True)

# Print number of outliers removed
print(f"Number of outliers removed: {number_of_rows_before_outliers - train_data.shape[0]}")
print(
    f"Percentage of rows removed: {((number_of_rows_before_outliers - train_data.shape[0]) / number_of_rows_before_outliers) * 100:.2f}%"
)

# Save number of outliers removed
number_of_outliers.loc["Number of Outliers", "None-iterative IQR"] = (
    number_of_rows_before_outliers - train_data.shape[0]
)
number_of_outliers.loc["Percentage of Outliers", "None-iterative IQR"] = round(
    ((number_of_rows_before_outliers - train_data.shape[0]) / number_of_rows_before_outliers) * 100, 2
)

##### Use pipelines


In [ ]:
# Fit and transform training data, transform test data
train_data_processed = complete_pipeline.fit_transform(train_data[all_columns], train_data["mwra"])
test_data_processed = complete_pipeline.transform(test_data[all_columns])

# Create DataFrames with selected feature names
train_data_processed = pd.DataFrame(train_data_processed, columns=transformed_feature_order)
test_data_processed = pd.DataFrame(test_data_processed, columns=transformed_feature_order)

# # Add mwra column back to the DataFrames
train_data_processed["mwra"] = train_data["mwra"]
test_data_processed["mwra"] = test_data["mwra"]

##### Export cleaned data


In [ ]:
os.makedirs("../data/clean_methods", exist_ok=True)

train_data_processed.to_csv("../data/clean_methods/train_1iqr.csv", index=False)
test_data_processed.to_csv("../data/clean_methods/test_1iqr.csv", index=False)

#### Iterative IQR


##### Cleaning


In [ ]:
# Load data
train_data, test_data = load_data_raw()

# Get number of rows before removing outliers
number_of_rows_before_outliers = train_data.shape[0]

# Define columns for IQR
columns_for_iqr = train_data.iloc[:, 3:].columns.difference(["p.android.vending"])

# Remove outliers using IQR method (in all columns)
train_data = remove_outliers_iqr(train_data, train_data.iloc[:, 3:].columns)

# Get number of outliers removed in next iteration
outliers_count = (
    ~(
        (
            train_data[columns_for_iqr]
            >= train_data[columns_for_iqr].quantile(0.25)
            - 1.5 * (train_data[columns_for_iqr].quantile(0.75) - train_data[columns_for_iqr].quantile(0.25))
        )
        & (
            train_data[columns_for_iqr]
            <= train_data[columns_for_iqr].quantile(0.75)
            + 1.5 * (train_data[columns_for_iqr].quantile(0.75) - train_data[columns_for_iqr].quantile(0.25))
        )
    ).all(axis=1)
).sum()

# Define maximum number of iterations
max_iterations = 10
iteration = 0

# Remove outliers iteratively
while outliers_count > 0:
    # Remove outliers using IQR method (in all columns except p.android.vending)
    train_data = remove_outliers_iqr(train_data, columns_for_iqr)

    # Get number of outliers removed in next iteration
    outliers_count = (
        ~(
            (
                train_data[columns_for_iqr]
                >= train_data[columns_for_iqr].quantile(0.25)
                - 1.5 * (train_data[columns_for_iqr].quantile(0.75) - train_data[columns_for_iqr].quantile(0.25))
            )
            & (
                train_data[columns_for_iqr]
                <= train_data[columns_for_iqr].quantile(0.75)
                + 1.5 * (train_data[columns_for_iqr].quantile(0.75) - train_data[columns_for_iqr].quantile(0.25))
            )
        ).all(axis=1)
    ).sum()

    # Increment iteration and stop if maximum number of iterations reached
    iteration += 1
    if iteration >= max_iterations:
        break

# Reset index
train_data = train_data.reset_index(drop=True)

# Print number of outliers removed
print(f"Number of outliers removed: {number_of_rows_before_outliers - train_data.shape[0]}")
print(
    f"Percentage of outliers removed: {((number_of_rows_before_outliers - train_data.shape[0]) / number_of_rows_before_outliers) * 100:.2f}%"
)

# Save number of outliers removed
number_of_outliers.loc["Number of Outliers", "Iterative IQR"] = (
    number_of_rows_before_outliers - train_data.shape[0]
)
number_of_outliers.loc["Percentage of Outliers", "Iterative IQR"] = round(
    ((number_of_rows_before_outliers - train_data.shape[0]) / number_of_rows_before_outliers) * 100, 2
)

##### Use pipelines


In [ ]:
# Fit and transform training data, transform test data
train_data_processed = complete_pipeline.fit_transform(train_data[all_columns], train_data["mwra"])
test_data_processed = complete_pipeline.transform(test_data[all_columns])

# Create DataFrames with selected feature names
train_data_processed = pd.DataFrame(train_data_processed, columns=transformed_feature_order)
test_data_processed = pd.DataFrame(test_data_processed, columns=transformed_feature_order)

# Add mwra column back to the DataFrames
train_data_processed["mwra"] = train_data["mwra"]
test_data_processed["mwra"] = test_data["mwra"]

##### Export processed data


In [ ]:
os.makedirs("../data/clean_methods", exist_ok=True)

train_data_processed.to_csv("../data/clean_methods/train_itiqr.csv", index=False)
test_data_processed.to_csv("../data/clean_methods/test_itiqr.csv", index=False)

#### None-iterative Z-score


##### Cleaning


In [ ]:
# Load data
train_data, test_data = load_data_raw()

# Get number of rows before removing outliers
number_of_rows_before_outliers = train_data.shape[0]

# Remove outliers using Z-score method
train_data = train_data[(np.abs(zscore(train_data.iloc[:, 3:])) < 3).all(axis=1)]
train_data = train_data.reset_index(drop=True)

print(f"Number of outliers removed: {number_of_rows_before_outliers - train_data.shape[0]}")
print(
    f"Percentage of outliers removed: {((number_of_rows_before_outliers - train_data.shape[0]) / number_of_rows_before_outliers) * 100:.2f}%"
)
number_of_outliers.loc["Number of Outliers", "None-iterative Z-score"] = (
    number_of_rows_before_outliers - train_data.shape[0]
)
number_of_outliers.loc["Percentage of Outliers", "None-iterative Z-score"] = round(
    ((number_of_rows_before_outliers - train_data.shape[0]) / number_of_rows_before_outliers) * 100, 2
)

##### Use pipelines


In [ ]:
# Fit and transform training data, transform test data
train_data_processed = complete_pipeline.fit_transform(train_data[all_columns], train_data["mwra"])
test_data_processed = complete_pipeline.transform(test_data[all_columns])

# Create DataFrames with selected feature names
train_data_processed = pd.DataFrame(train_data_processed, columns=transformed_feature_order)
test_data_processed = pd.DataFrame(test_data_processed, columns=transformed_feature_order)

# Add mwra column back to the DataFrames
train_data_processed["mwra"] = train_data["mwra"]
test_data_processed["mwra"] = test_data["mwra"]

##### Export processed data


In [ ]:
os.makedirs("../data/clean_methods", exist_ok=True)

train_data_processed.to_csv("../data/clean_methods/train_1zscore.csv", index=False)
test_data_processed.to_csv("../data/clean_methods/test_1zscore.csv", index=False)

#### Iterative Z-score


##### Cleaning


In [ ]:
train_data, test_data = load_data_raw()

# 1 iteration of cleaning whole dataset of outliers (including p.android.vending)
train_data = train_data[(np.abs(zscore(train_data.iloc[:, 3:])) < 3).all(axis=1)]

# Using all columns except c.android.vending for outlier detection
columns_for_zscore = train_data.iloc[:, 3:].columns.difference(["p.android.vending"])
outliers_count = (~(np.abs(zscore(train_data[columns_for_zscore])) < 3).all(axis=1)).sum()
max_iterations = 10
iteration = 0

# Iterating after we removed all outliers
while outliers_count > 0:
    train_data = train_data[(np.abs(zscore(train_data[columns_for_zscore])) < 3).all(axis=1)]
    outliers_count = (~(np.abs(zscore(train_data[columns_for_zscore])) < 3).all(axis=1)).sum()
    iteration += 1
    if iteration >= max_iterations:
        break

train_data = train_data.reset_index(drop=True)

print(f"Number of outliers removed: {number_of_rows_before_outliers - train_data.shape[0]}")
print(
    f"Percentage of outliers removed: {((number_of_rows_before_outliers - train_data.shape[0]) / number_of_rows_before_outliers) * 100:.2f}%"
)
number_of_outliers.loc["Number of Outliers", "Iterative Z-score"] = (
    number_of_rows_before_outliers - train_data.shape[0]
)
number_of_outliers.loc["Percentage of Outliers", "Iterative Z-score"] = round(
    ((number_of_rows_before_outliers - train_data.shape[0]) / number_of_rows_before_outliers) * 100, 2
)

##### Use pipelines


In [ ]:
# Fit and transform training data, transform test data
train_data_processed = complete_pipeline.fit_transform(train_data[all_columns], train_data["mwra"])
test_data_processed = complete_pipeline.transform(test_data[all_columns])

# Create DataFrames with selected feature names
train_data_processed = pd.DataFrame(train_data_processed, columns=transformed_feature_order)
test_data_processed = pd.DataFrame(test_data_processed, columns=transformed_feature_order)

# Add mwra column back to the DataFrames
train_data_processed["mwra"] = train_data["mwra"]
test_data_processed["mwra"] = test_data["mwra"]

##### Export processed data


In [ ]:
os.makedirs("../data/clean_methods", exist_ok=True)

train_data_processed.to_csv("../data/clean_methods/train_itzscore.csv", index=False)
test_data_processed.to_csv("../data/clean_methods/test_itzscore.csv", index=False)

### Show comparison


In [ ]:
number_of_outliers

### Using on Models


#### None-iterative IQR


In [ ]:
# Load data
train_data_processed = pd.read_csv("../data/clean_methods/train_1iqr.csv")
test_data_processed = pd.read_csv("../data/clean_methods/test_1iqr.csv")

# Train data without feature selection
X_train = train_data_processed.drop(columns=["mwra"])
y_train = train_data_processed["mwra"]

# Test data without feature selection
X_test = test_data_processed.drop(columns=["mwra"])
y_test = test_data_processed["mwra"]

In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    n_estimators=300, max_depth=15, min_samples_split=10, min_samples_leaf=1, ccp_alpha=0.001, random_state=42
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
scores_1iqr = get_scores(rf_classifier, "1 IQR", X_train, y_train, X_test, y_test)

#### Iterative IQR


In [ ]:
# Load data
train_data_processed = pd.read_csv("../data/clean_methods/train_itiqr.csv")
test_data_processed = pd.read_csv("../data/clean_methods/test_itiqr.csv")

# Train data without feature selection
X_train = train_data_processed.drop(columns=["mwra"])
y_train = train_data_processed["mwra"]

# Test data without feature selection
X_test = test_data_processed.drop(columns=["mwra"])
y_test = test_data_processed["mwra"]

In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    n_estimators=300, max_depth=15, min_samples_split=10, min_samples_leaf=1, ccp_alpha=0.001, random_state=42
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
scores_itiqr = get_scores(rf_classifier, "Iterative IQR", X_train, y_train, X_test, y_test)

#### None-iterative Z-score


In [ ]:
# Load data
train_data_processed = pd.read_csv("../data/clean_methods/train_1zscore.csv")
test_data_processed = pd.read_csv("../data/clean_methods/test_1zscore.csv")

# Train data without feature selection
X_train = train_data_processed.drop(columns=["mwra"])
y_train = train_data_processed["mwra"]

# Test data without feature selection
X_test = test_data_processed.drop(columns=["mwra"])
y_test = test_data_processed["mwra"]

In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    n_estimators=300, max_depth=15, min_samples_split=10, min_samples_leaf=1, ccp_alpha=0.001, random_state=42
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
scores_1zscore = get_scores(rf_classifier, "1 Z-score", X_train, y_train, X_test, y_test)

#### Iterative Z-score


In [ ]:
# Load data
train_data_processed = pd.read_csv("../data/clean_methods/train_itzscore.csv")
test_data_processed = pd.read_csv("../data/clean_methods/test_itzscore.csv")

# Train data without feature selection
X_train = train_data_processed.drop(columns=["mwra"])
y_train = train_data_processed["mwra"]

# Test data without feature selection
X_test = test_data_processed.drop(columns=["mwra"])
y_test = test_data_processed["mwra"]

In [ ]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(
    n_estimators=300, max_depth=15, min_samples_split=10, min_samples_leaf=1, ccp_alpha=0.001, random_state=42
)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Print the scores
scores_itzscore = get_scores(rf_classifier, "Iterative Z-score", X_train, y_train, X_test, y_test)

### Show comparison


In [ ]:
scores = pd.concat([scores_1iqr, scores_itiqr, scores_1zscore, scores_itzscore], axis=1)
scores

## B


Dátová transformácia (scaling, transformer, ...).

-   In previous step, we found that Iterative Z-score outlier detection gives better results, so we are going to use this method in this step.


### Definitions


In [ ]:
def load_data_clean():
    file_path: str = "../data/clean_methods"

    # Load cleaned data (Iterative Z-score)
    train_data = pd.read_csv(f"{file_path}/train_itzscore.csv")
    test_data = pd.read_csv(f"{file_path}/test_itzscore.csv")

    # Define train & test data
    X_train = train_data.drop(columns=["mwra"])
    y_train = train_data["mwra"]

    X_test = test_data.drop(columns=["mwra"])
    y_test = test_data["mwra"]

    return X_train, y_train, X_test, y_test

In [ ]:
# Define columns
gaussian_columns = [
    "c.dogalize",
    "c.android.gm",
    "c.android.youtube",
    "c.android.chrome",
    "c.katana",
    "c.raider",
    "p.android.packageinstaller",
    "p.android.settings",
    "p.android.documentsui",
    "p.android.chrome",
    "p.android.gm",
    "p.system",
    "p.android.externalstorage",
    "p.process.gapps",
    "p.google",
    "p.browser.provider",
    "p.android.defcontainer",
]

log_columns = ["c.android.vending"]

uniform_columns = [
    "c.UCMobile.x86",
    "c.updateassist",
    "c.UCMobile.intl",
    "p.android.vending",
    "p.dogalize",
    "p.olauncher",
    "p.simulator",
    "p.inputmethod.latin",
    "p.android.gms",
    "p.notifier",
    "p.katana",
    "p.gms.persistent",
]


transformed_feature_order = pd.Series(gaussian_columns + log_columns + uniform_columns)

### Define different pipelines


#### Our pipeline from 2nd Phase


In [ ]:
gaussian_pipeline = Pipeline(
    [
        ("scaler", RobustScaler()),
        ("power_transformer", PowerTransformer(method="yeo-johnson")),
    ]
)

non_gaussian_pipeline = Pipeline(
    [
        ("scaler", RobustScaler()),
        ("quantile_transformer", QuantileTransformer(output_distribution="normal", random_state=42)),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("general", gaussian_pipeline, gaussian_columns),
        ("vending", non_gaussian_pipeline, non_gaussian_columns),
    ],
    remainder="passthrough",
)

robust_pipeline = Pipeline([("preprocessor", preprocessor)])

#### Defining other pipelines


In [ ]:
# Pipeline 1: Basic scaling
basic_pipeline = Pipeline([("scaler", StandardScaler())])

# Pipeline 2: Robust scaling for outlier handling
robust_pipeline = Pipeline([("scaler", RobustScaler()), ("power", PowerTransformer(method="yeo-johnson"))])

# Pipeline 3: Quantile transformation
quantile_pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("quantile", QuantileTransformer(output_distribution="normal", n_quantiles=1000)),
    ]
)

# Pipeline 4: Min-max scaling with power transform
minmax_pipeline = Pipeline([("scaler", MinMaxScaler()), ("power", PowerTransformer(method="yeo-johnson"))])

# Pipeline 5: Log transform for skewed data
log_pipeline = Pipeline([("log", FunctionTransformer(func=np.log1p)), ("scaler", StandardScaler())])


# Create column transformers for different data types
def create_full_pipeline(gaussian_pipe, uniform_pipe, log_pipe):
    return ColumnTransformer(
        [
            ("gaussian", gaussian_pipe, gaussian_columns),
            ("uniform", uniform_pipe, uniform_columns),
            ("log", log_pipe, log_columns),
        ]
    )


# Create different combinations to test
pipeline_combinations = [
    # Combination 1: Basic scaling for all
    create_full_pipeline(basic_pipeline, basic_pipeline, basic_pipeline),
    # Combination 2: Robust handling
    create_full_pipeline(robust_pipeline, robust_pipeline, robust_pipeline),
    # Combination 3: Type-specific transforms
    create_full_pipeline(basic_pipeline, quantile_pipeline, log_pipeline),
    # Combination 4: Quantile for all
    create_full_pipeline(quantile_pipeline, quantile_pipeline, quantile_pipeline),
    # Combination 5: MinMax with power transforms
    create_full_pipeline(minmax_pipeline, minmax_pipeline, minmax_pipeline),
]


# Function to evaluate pipeline performance
def evaluate_pipeline(pipeline, X_train, X_test, y_train, y_test):
    pipeline.fit(X_train, y_train)
    train_score = pipeline.score(X_train, y_train)
    test_score = pipeline.score(X_test, y_test)
    return train_score, test_score

## C


Výber atribútov, výber algoritmov, hyperparameter tuning, ensemble learning.


## D


Ktorý model je Váš najlepší model pre nasadenie (deployment)?


## E


Aký je data pipeline pre jeho vybudovanie na základe Vášho datasetu v produkcii?
